In [1]:
import imutils
from imutils.video import VideoStream, FileVideoStream
from imutils import face_utils
import cv2
import time
import numpy as np
import dlib
from collections import OrderedDict

# Face streamer

In [2]:
# Face streamer class 
class face_streamer:
    def __init__(self, predictor_path, filename = None):
        self.filename = filename
        
        # Initialize dlib's face detector (HOG-based)
        self.detector = dlib.get_frontal_face_detector()
        
        # Create landmark predictor.
        self.predictor = dlib.shape_predictor("../detect-face-parts/shape_predictor_68_face_landmarks.dat")
        
        # Facial landmarks that we use
        facial_landmarks_idxs = OrderedDict([
            ("nose", (27, 36)),
            ("face", (0, 26))
        ])
        
    def start_stream(self):
        if self.filename:
            self.vs = FileVideoStream(self.filename).start()
        else:
            self.vs = VideoStream(src=0).start()
        print("[INFO] camera sensor warming up...")
        time.sleep(2.0)
            
    def end_stream(self):
        # Do some cleanup
        cv2.destroyAllWindows()
        self.vs.stop()
    
    def stream(self, display_face_bb = False, display_landmarks = False):
        self.start_stream()
        # Loop and stream
        while True:
            # Read and resize the frame
            frame = self.vs.read()
            frame = imutils.resize(frame, width=400)
            # Get grayscale image and extract the bounding boxes with the detector 
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            rects = self.detector(gray, 0)
            # Loop over the face detections 
            for rect in rects:
                # Display bounding box if true
                if display_face_bb:
                    frame = self.display_face_bb(frame, rect)
                # Determine the facial landmarks for the face region, then
                # convert the facial landmark (x, y)-coordinates to a NumPy
                # array.
                shape = face_utils.shape_to_np(self.predictor(gray, rect))
                # Display facial landmarks if true
                if display_landmarks:
                    frame = self.display_landmarks(frame, rect)
                
            # Show the frame
            cv2.imshow("Frame", frame)
            key = cv2.waitKey(1) & 0xFF
            
            # If the `q` key was pressed, break from the loop.
            if key == ord("q"):
                # Do some cleanup 
                self.end_stream()
                del self.vs
                break
    
    def display_face_bb(self, frame, rect):
        # Compute the bounding box of the face and draw it on the
        # frame
        (bX, bY, bW, bH) = face_utils.rect_to_bb(rect)
        cv2.rectangle(frame, (bX, bY), (bW+bX, bH+bY), (0, 255, 0), 1)
        return frame
    
    def display_landmarks(self, frame, rect):
        for (name, (i, j)) in self.facial_landmarks_idxs.items():
            # Loop over the subset of facial landmarks, drawing the specific face part
            for (x, y) in shape[i:j]:
                cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
        return frame 
        

    def draw_aam(self):
        return
                
# next

In [3]:
FACIAL_LANDMARKS_IDXS = OrderedDict([
    ("nose", (27, 36)),
    ("face", (0, 26))
])
predictor_path = "../detect-face-parts/shape_predictor_68_face_landmarks.dat"
fs = face_streamer(predictor_path)
fs.stream(display_face_bb = False, display_landmarks = True)
# del fs

[INFO] camera sensor warming up...


AttributeError: 'face_streamer' object has no attribute 'facial_landmarks_idxs'

In [ ]:
def stream_face_detection(filename=None):
    # To capture video from webcam.
    if not _filename:
        vs = VideoStream(src=0).start()
    else:
        # To use a video file as input 
        vs = FileVideoStream(filename).start()
    time.sleep(2.0)

    while True:
        # grab frame and resize
        frame = vs.read()
        frame = imutils.resize(frame, width=400)
        # grab the frame dimensions and convert it to a blob
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
            (300, 300), (104.0, 177.0, 123.0))
        # pass the blob through the network and obtain the detections and
        # predictions
        net.setInput(blob)
        detections = net.forward()
        for i in range(0, detections.shape[2]):
            # extract the confidence (i.e., probability) associated with the
            # prediction
            confidence = detections[0, 0, i, 2]
            # filter out weak detections by ensuring the `confidence` is
            # greater than the minimum confidence
            if confidence < confidence_threshold:
                continue
            # compute the (x, y)-coordinates of the bounding box for the
            # object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # draw the bounding box of the face along with the associated
            # probability
            text = "{:.2f}%".format(confidence * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                (0, 0, 255), 2)
            cv2.putText(frame, text, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

        # show the output frame
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF

        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            # do a bit of cleanup
            cv2.destroyAllWindows()
            vs.stop()
            break

In [ ]:
def show_image(_path):
    img = cv2.imread(_path,1)
    while True:
        cv2.imshow('image',img)
        key = cv2.waitKey(1) & 0xFF
        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            # do a bit of cleanup
            cv2.destroyAllWindows()
            break